# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 215.7MB 56kB/s 
     |████████████████████████████████| 204kB 52.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=0e4fc5fcc2d3738b959890997ef0d88ebc3e51efb0bdec012e577cec1e377cd2
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u232-b09-0ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-01-21 00:18:14--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.237.203.145, 3.223.119.4, 34.202.58.243, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.237.203.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  13.9MB/s    in 0.9s    

2020-01-21 00:18:16 (13.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://2c19c20e.ngrok.io


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [0]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [10]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [11]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [57]:
# YOUR CODE HERE
data = products.join(orders, products.product_id == orders.product_id).groupBy('order_id').agg(collect_list("product_name").alias("products_name"))
data.show()

+--------+--------------------+
|order_id|       products_name|
+--------+--------------------+
|    1342|[Bag of Organic B...|
|    1591|[Cracked Wheat, O...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    6357|[Fresh Mozzarella...|
|   10362|[Crinkle Cut Fren...|
|   19204|[Lemon Lime Thirs...|
|   29601|[Toasted Coconut ...|
|   31035|[Bag of Organic B...|
|   40011|[Sea Salt Macadam...|
|   46266|[Organic Uncured ...|
|   51607|[Major Dickason's...|
|   58797|[Bag of Organic B...|
|   61793|[Raspberries, Nat...|
|   67089|[Banana, Organic ...|
|   70863|[Bathroom Tissue,...|
|   88674|[Unsweetened Almo...|
|   91937|[No. 485 Gin, Tra...|
|   92317|[Banana, Indian M...|
|   99621|[Organic Basil, F...|
+--------+--------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [0]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="products_name", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(data)

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [62]:
# YOUR CODE HERE
freqItemsets = model.freqItemsets
associationRules = model.associationRules

print(freqItemsets.count())
print(associationRules.count())

120
0


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [63]:
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products_name", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(data)

print(model.freqItemsets.count())
print(model.associationRules.count())

4444
11


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [64]:
# YOUR CODE HERE
freqItemsets = model.freqItemsets
freqItemsets.sort(desc("freq")).take(20)

[Row(items=['Banana'], freq=18726),
 Row(items=['Bag of Organic Bananas'], freq=15480),
 Row(items=['Organic Strawberries'], freq=10894),
 Row(items=['Organic Baby Spinach'], freq=9784),
 Row(items=['Large Lemon'], freq=8135),
 Row(items=['Organic Avocado'], freq=7409),
 Row(items=['Organic Hass Avocado'], freq=7293),
 Row(items=['Strawberries'], freq=6494),
 Row(items=['Limes'], freq=6033),
 Row(items=['Organic Raspberries'], freq=5546),
 Row(items=['Organic Blueberries'], freq=4966),
 Row(items=['Organic Whole Milk'], freq=4908),
 Row(items=['Organic Cucumber'], freq=4613),
 Row(items=['Organic Zucchini'], freq=4589),
 Row(items=['Organic Yellow Onion'], freq=4290),
 Row(items=['Organic Garlic'], freq=4158),
 Row(items=['Seedless Red Grapes'], freq=4059),
 Row(items=['Asparagus'], freq=3868),
 Row(items=['Organic Grape Tomatoes'], freq=3823),
 Row(items=['Organic Red Onion'], freq=3818)]

In [55]:
model.associationRules.take(20)

[Row(antecedent=['Organic Kiwi', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5459770114942529, lift=4.627719489738336),
 Row(antecedent=['Organic Raspberries', 'Organic Hass Avocado', 'Organic Strawberries'], consequent=['Bag of Organic Bananas'], confidence=0.5984251968503937, lift=5.072272070642333),
 Row(antecedent=['Organic Broccoli', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5048231511254019, lift=4.278897986822536),
 Row(antecedent=['Organic Unsweetened Almond Milk', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5141065830721003, lift=4.357584667849303),
 Row(antecedent=['Yellow Onions', 'Strawberries'], consequent=['Banana'], confidence=0.5357142857142857, lift=3.7536332219526702),
 Row(antecedent=['Organic Cucumber', 'Organic Hass Avocado', 'Organic Strawberries'], consequent=['Bag of Organic Bananas'], confidence=0.546875, lift=4.635330870478036),
 Row(antecedent=['Organic Navel 

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!